# Notebook 03: OpenMC Loop & Inference

## Solving the Validation Paradox

**Learning Objective:** Validate models using reactor simulations and sensitivity weighting.

### The Validation Paradox (Revisited)

```
Model A: Test MSE = 0.010, k_eff error = 450 pcm → UNSAFE!
Model B: Test MSE = 0.015, k_eff error = 50 pcm  → SAFE!
```

**Standard ML** picks Model A (lower MSE)  
**Physics-informed ML** picks Model B (better reactor prediction)

### The Solution: Sensitivity Weighting

Weight loss by ∂k/∂σ:
- High-sensitivity reactions (U-235 fission) → Large penalty
- Low-sensitivity reactions (trace captures) → Small penalty

In [ ]:
import sys
sys.path.append('..')

import numpy as np
from pathlib import Path
from nucml_next.validation import OpenMCValidator, SensitivityAnalyzer, ReactorBenchmark

# Verify EXFOR data exists (needed for model training/validation)
exfor_path = Path('../data/exfor_processed.parquet')
if not exfor_path.exists():
    raise FileNotFoundError(
        f"EXFOR data not found at {exfor_path}\n"
        "Please run: python scripts/ingest_exfor.py --exfor-root <path> --output data/exfor_processed.parquet"
    )

print("✓ Imports successful")
print("✓ EXFOR data found")

### Step 1: Sensitivity Analysis

In [ ]:
# Analyze which reactions matter for reactors
analyzer = SensitivityAnalyzer(reactor_type='PWR')
analyzer.print_sensitivity_report()

print("\n💡 Key Insight: U-235 fission has 50× higher sensitivity than minor reactions!")
print("   Errors in U-235 fission are 50× more dangerous for reactor safety.")

### Step 2: Validate with OpenMC

In [ ]:
# Initialize validator
validator = OpenMCValidator(reactor_type='PWR')

# Simulate k_eigenvalue with model predictions
# NOTE: In production, these would come from your trained GNN-Transformer model
# trained on real EXFOR data. Here we demonstrate the validation workflow.
cross_section_data = {
    (92, 235, 18): np.array([500.0]),  # U-235 fission (example prediction)
    (92, 235, 102): np.array([100.0]), # U-235 capture (example prediction)
}

results = validator.run_k_eigenvalue(cross_section_data)

print(f"\nReactor Prediction:")
print(f"  k_eff = {results['k_eff_mean']:.5f} ± {results['k_eff_std']:.5f}")
print(f"  Reference = {results['k_eff_nominal']:.5f}")

# Compute reactivity error
error_pcm = validator.compute_reactivity_error(results['k_eff_mean'])
print(f"  Error = {error_pcm:.1f} pcm")

if abs(error_pcm) < 100:
    print("\n✓ EXCELLENT reactor prediction!")
elif abs(error_pcm) < 300:
    print("\n✓ ACCEPTABLE reactor prediction")
else:
    print("\n❌ POOR reactor prediction - needs improvement")

### Step 3: Benchmark Validation

In [ ]:
# Compare against standard reactor benchmarks
ReactorBenchmark.list_available_benchmarks()

# Validate against PWR pin cell
benchmark = ReactorBenchmark('PWR_PIN')
benchmark.print_benchmark_info()

# Validate prediction
validation = benchmark.validate_prediction(results['k_eff_mean'])
benchmark.print_validation_results(validation)

### Step 4: Retrain with Sensitivity Weighting

In [ ]:
from nucml_next.physics import SensitivityWeightedLoss

# Get sensitivity weights
weights = analyzer.get_sensitivity_weights(cross_section_data)

# Create sensitivity-weighted loss
loss_fn = SensitivityWeightedLoss()

print("\n🎯 Training Strategy:")
print("   1. Train with standard MSE → Get baseline")
print("   2. Retrain with sensitivity weighting → Improve reactor accuracy")
print("   3. Validate with OpenMC → Verify k_eff error < 100 pcm")
print("\n   This is how we solve the Validation Paradox!")

### 🎓 Final Takeaway

> **The Validation Paradox is SOLVED!**
>
> By combining real EXFOR data with sensitivity-weighted validation:
> 1. We train on experimental measurements (not simulations)
> 2. We prioritize safety-critical reactions via sensitivity weighting
> 3. We validate with reactor simulations (OpenMC)
> 4. We achieve reactor-safe ML predictions
>
> **Low MSE ≠ Safe Reactor**  
> **Low Sensitivity-Weighted Error + Real Data = Safe Reactor** ✓

---

## Summary: The NUCML-Next Journey

| Notebook | Topic | Key Lesson |
|----------|-------|------------|
| 00 | Baselines | Classical ML fails (staircase effect) on real data |
| 01 | Data Fabric | Nuclear data = Graph representation |
| 02 | GNN-Transformer | Deep learning → Smooth predictions from real EXFOR |
| 03 | OpenMC Validation | Reactor safety > Geometric MSE |

### What You've Learned

✓ Why classical ML fails for physics problems  
✓ How to build graph representations from real nuclear data  
✓ How to train physics-informed models on EXFOR measurements  
✓ How to validate with reactor simulations  
✓ How to solve the Validation Paradox with sensitivity weighting  

---

**Congratulations! You've completed NUCML-Next!** 🎉

*You're now ready to apply physics-informed deep learning to production nuclear data evaluation using real IAEA EXFOR experimental measurements.*